<a href="https://colab.research.google.com/github/ysj9909/DL_practice_from_scratch/blob/main/LSTM_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MNIST classification using LSTM from scratch - code practice**

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Hyper-parameters
num_epochs = 5
learning_rate = 0.0003
batch_size = 100

In [ ]:
data_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5), (0.5))])

train_dataset = torchvision.datasets.MNIST(root = "./data", train = True, download = True, transform = data_transforms)
test_dataset = torchvision.datasets.MNIST(root = "./data", train = False, transform = data_transforms)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
class LSTM_Classification_Model(nn.Module):
  def __init__(self, input_dim = 28, hidden_dim = 128, sequence_length = 28, num_classes = 10):
    super(LSTM_Classification_Model, self).__init__()
    
    self.sequence_length = sequence_length
    self.hidden_dim = hidden_dim

    self.W_f = nn.Linear(input_dim + hidden_dim, hidden_dim)
    self.W_i = nn.Linear(input_dim + hidden_dim, hidden_dim)
    self.W_o = nn.Linear(input_dim + hidden_dim, hidden_dim)
    self.W_C = nn.Linear(input_dim + hidden_dim, hidden_dim)

    self.fc_output = nn.Linear(hidden_dim, num_classes)
    self.tanh = nn.Tanh()

  def forward(self, imgs):
    # imgs : (batch_size, 1, sequence_length, input_dim)
    h = torch.zeros(imgs.size(0), self.hidden_dim)
    c = torch.zeros(imgs.size(0), self.hidden_dim)
    for i in range(self.sequence_length):
      input = torch.cat([h, imgs[:,0, i, :]], dim = -1)   # (batch_size, input_dim + hidden_dim)

      input_gate = torch.sigmoid(self.W_i(input))
      forget_gate = torch.sigmoid(self.W_f(input))
      output_gate = torch.sigmoid(self.W_o(input))

      C_tilda = self.tanh(self.W_C(input))
      c = forget_gate * c + input_gate * C_tilda
      h = output_gate * self.tanh(c)
    
    outputs = self.fc_output(h)

    return outputs

In [ ]:
model  = LSTM_Classification_Model()

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# Train the model
for epoch in range(num_epochs):
  for i, (imgs, labels) in enumerate(train_loader):
    outputs  = model(imgs)

    optimizer.zero_grad()
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      print(f"Epoch [{epoch + 1} / {num_epochs}], Step [{i + 1} / {len(train_loader)}], Loss : {loss.item()}")


Epoch [1 / 5], Step [100 / 600], Loss : 0.06972099095582962
Epoch [1 / 5], Step [200 / 600], Loss : 0.050710421055555344
Epoch [1 / 5], Step [300 / 600], Loss : 0.049098461866378784
Epoch [1 / 5], Step [400 / 600], Loss : 0.02276918664574623
Epoch [1 / 5], Step [500 / 600], Loss : 0.03190048038959503
Epoch [1 / 5], Step [600 / 600], Loss : 0.07968086004257202
Epoch [2 / 5], Step [100 / 600], Loss : 0.025075728073716164
Epoch [2 / 5], Step [200 / 600], Loss : 0.12323637306690216
Epoch [2 / 5], Step [300 / 600], Loss : 0.08443937450647354
Epoch [2 / 5], Step [400 / 600], Loss : 0.02667210064828396
Epoch [2 / 5], Step [500 / 600], Loss : 0.0635746419429779
Epoch [2 / 5], Step [600 / 600], Loss : 0.05376444756984711
Epoch [3 / 5], Step [100 / 600], Loss : 0.036978911608457565
Epoch [3 / 5], Step [200 / 600], Loss : 0.08832263946533203
Epoch [3 / 5], Step [300 / 600], Loss : 0.10294365882873535
Epoch [3 / 5], Step [400 / 600], Loss : 0.04448170214891434
Epoch [3 / 5], Step [500 / 600], Loss

In [ ]:
# Test the model
with torch.no_grad():
  correct = 0
  total = 0
  for imgs, labels in test_loader:

    outputs = model(imgs)
    _, predicted = torch.max(outputs, dim = -1)

    correct += (predicted == labels).sum().item()
    total += imgs.size(0)

  print(f"Accuracy of the model on the test data : {round(100 * correct / total, 3)} ")



Accuracy of the model on the test data : 98.38 
